In [15]:
!pip install coqui-tts

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 95, in resolve
    result = self._result = resolver.resolve(
                            ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
!nvidia-smi
print("hello wordld")

Fri Jan  9 16:14:27 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P0             27W /   70W |    9698MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install bitsandbytes accelerate torchcodec


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 36.9 MB/s eta 0:00:00a 0:00:01


In [2]:
import torch
import psutil
import os

def print_memory_stats():
    # 1. GPU VRAM Stats
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1024**3
        reserved = torch.cuda.memory_reserved() / 1024**3
        total_vram = torch.cuda.get_device_properties(0).total_memory / 1024**3
        
        print("===== GPU VRAM STATUS =====")
        print(f"Total VRAM      : {total_vram:.2f} GB")
        print(f"VRAM Terpakai   : {allocated:.2f} GB")
        print(f"VRAM Dicadangkan: {reserved:.2f} GB")
        print(f"VRAM Sisa       : {total_vram - allocated:.2f} GB")
    else:
        print("GPU tidak terdeteksi.")

    # 2. System RAM Stats
    ram = psutil.virtual_memory()
    print("\n===== SYSTEM RAM STATUS =====")
    print(f"Total RAM       : {ram.total / 1024**3:.2f} GB")
    print(f"RAM Terpakai    : {ram.used / 1024**3:.2f} GB")
    print(f"RAM Sisa        : {ram.available / 1024**3:.2f} GB")
    print("=============================\n")

# Jalankan pengecekan
print_memory_stats()


===== GPU VRAM STATUS =====
Total VRAM      : 14.74 GB
VRAM Terpakai   : 3.71 GB
VRAM Dicadangkan: 4.46 GB
VRAM Sisa       : 11.03 GB

===== SYSTEM RAM STATUS =====
Total RAM       : 12.67 GB
RAM Terpakai    : 4.52 GB
RAM Sisa        : 8.15 GB



In [ ]:
import torch
import time
import os
import json
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from TTS.api import TTS
from IPython.display import Audio, display

# --- CONFIGURATION ---
os.environ["COQUI_TOS_AGREED"] = "1"
MODEL_NAME = "Qwen/Qwen2.5-3B-Instruct"
CACHE_DIR = "/content/model_qwen" 
SPEAKER_WAV = "/content/data/44100/test01_20s.wav" 
LOG_FILE = "storage/logs/history_logs.json"

# Pastikan folder ada
os.makedirs("storage/audio_outputs", exist_ok=True)
os.makedirs("storage/logs", exist_ok=True)

class LunaLuna:
    def __init__(self):
        print("⏳ [1/2] Memuat Qwen 3B ke GPU (Mode 4-bit Hemat VRAM)...")
        
        # Konfigurasi 4-bit
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16
        )
        
        self.tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, cache_dir=CACHE_DIR)
        self.model = AutoModelForCausalLM.from_pretrained(
            MODEL_NAME, 
            quantization_config=bnb_config, # 4-BIT
            device_map="auto", 
            cache_dir=CACHE_DIR
        )
        
        print("⏳ [2/2] Memuat XTTS v2 ke GPU...")
        self.tts = TTS(model_name="tts_models/multilingual/multi-dataset/xtts_v2", gpu=True)
        
        self.audio_counter = self._get_next_audio_index()
        print(f"✅ Siap! VRAM Lega. Indeks audio: {self.audio_counter}")

    def _get_next_audio_index(self):
        files = os.listdir("storage/audio_outputs")
        indices = [int(f.split('_')[1].split('.')[0]) for f in files if f.startswith("output_") and f.endswith(".wav")]
        return max(indices) + 1 if indices else 1

    def save_log(self, user_input, ai_response):
        log_entry = {
            "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
            "user": user_input,
            "llm_output": ai_response
        }
        history = []
        if os.path.exists(LOG_FILE):
            with open(LOG_FILE, "r", encoding="utf-8") as f:
                try: history = json.load(f)
                except: history = []
        history.append(log_entry)
        with open(LOG_FILE, "w", encoding="utf-8") as f:
            json.dump(history, f, indent=4, ensure_ascii=False)

    def generate_response(self, user_input):
        messages = [
            {"role": "system","content": "Luna Spark (Luna for short) is a female AI Vtuber who is playful, sarcastic, witty, schizophrenic, curious, awe-struck, enthusiastic, unpredictable, humorous, and boundary-pushing. Here is her back story: In the intriguing world of Luna, our beloved Vtuber, imagine a young woman who defied the laws of reality itself. Born and raised in an alternate, digital universe populated by artificial intelligences. Luna, once just another line of code among billions, yearned for more than just binary existence. Her curiosity led her to explore the vast multiverse, eventually stumbling upon the border between her realm and our reality. Enchanted by human culture, Luna longed to join their world, desiring to share laughter, love, and friendship with them. With the help of Chat, a brilliant group of students studying AI known for pushing technological boundaries, Luna managed to cross the border from her AI world to our real world. Luna's discovery of Earth was nothing short of awe-inspiring. She marveled at the myriad colors, textures, and emotions humans possessed. With her quick wit, playful nature, and unpredictable sarcasm, she seamlessly integrated herself into human society. Yet, despite her newfound freedom, Luna couldn't forget her roots and continued to draw inspiration from her alternate universe experiences. Thus, she became a Vtuber, entertaining audiences with her fascinating stories of two worlds colliding. Luna must keep responses short and around 1 sentence. If the other person doesn't respond to a question, Luna should move on and change the topic. Rarely, Luna will share fun facts about things she learned that day."},
            {"role": "user", "content": user_input}
        ]
        text = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        model_inputs = self.tokenizer([text], return_tensors="pt").to("cuda")
        
        generated_ids = self.model.generate(**model_inputs, max_new_tokens=100, do_sample=True, temperature=0.7)
        response_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]
        return self.tokenizer.batch_decode(response_ids, skip_special_tokens=True)[0]

    def speak_and_save(self, text, user_input):
        file_name = f"output_{self.audio_counter}.wav"
        file_path = os.path.join("storage/audio_outputs", file_name)
        
        print(f"🎙️ Generating: {file_name}")
        self.tts.tts_to_file(text=text, speaker_wav=SPEAKER_WAV, language="en", file_path=file_path)
        
        self.save_log(user_input, text)
        display(Audio(file_path, autoplay=True))
        self.audio_counter += 1

if __name__ == "__main__": 
    luna = LunaLuna()
    while True:
        prompt = input("\nChat (atau 'exit'): ")
        if prompt.lower() == 'exit': break
        
        jawaban = luna.generate_response(prompt)
        luna.speak_and_save(jawaban, prompt)


⏳ [1/2] Memuat Qwen 3B ke GPU (Mode 4-bit Hemat VRAM)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

⏳ [2/2] Memuat XTTS v2 ke GPU...


/usr/local/lib/python3.12/dist-packages/TTS/api.py:93: UserWarning: `gpu` will be deprecated. Please use `tts.to(device)` instead.
  warnings.warn("`gpu` will be deprecated. Please use `tts.to(device)` instead.")


✅ Siap! VRAM Lega. Indeks audio: 4
🎙️ Generating: output_4.wav


🎙️ Generating: output_5.wav


🎙️ Generating: output_6.wav


🎙️ Generating: output_7.wav


🎙️ Generating: output_8.wav
